# Table-of-content  

[Packages](#Import-Packages)  
[AOI](#AOI)  
[Bare Soil](#Bare-Soil)  
[Corine Land Cover](#Corine-Land-Cover-(CLC))  
[Precipitaion Data](#Precipitaion-Data)  
[Soil Moisture](#Soil-Moisture)  
[Sentinel1 derives soil-moisture](#Sentinel1-derives-soil-moisture)    
[Get CSV for Datasets](#Get-CSV-for-Datasets)  
[Data Visualisation](#Data-Visualisation)  
[Collection of commands](#Collection-of-commands)  
[Collection of Notebook snippets](#Collection-of-Notebook-snippets)

# Earth Engine Communicator 
from-gee-to-numpy-to-geotiff https://mygeoblog.com/2017/10/06/from-gee-to-numpy-to-geotiff/  
API DOC: https://developers.google.com/earth-engine/api_docs#eefeatureget  
ReduceRegion https://developers.google.com/earth-engine/reducers_reduce_region  
Scale https://developers.google.com/earth-engine/scale  
Histogram Matching https://gis.stackexchange.com/questions/332121/histogram-matching-in-google-earth-engine    
Determining the sentinel 1 tracks https://gis.stackexchange.com/questions/286922/determining-if-sentinel-1-orbit-is-ascending-or-descending-from-absolute-orbit-n    
create sample from feature collection https://mygeoblog.com/2019/04/03/create-sample-from-feature-collection/    
Jupter EE map https://github.com/spadarian/jupyter_ee_map    
Tutorials https://github.com/csaybar/EEwPython
TimeSeries and other nice Stuff https://github.com/tylere/PyDataNYC2017/blob/master/ipynb/satellite_analysis.ipynb
https://eeconvert.readthedocs.io/en/latest/eeconvert.html  
https://mygeoblog.com/2019/08/21/google-earth-engine-to-numpy/  

TO-DO: change ReduceRegion function by adding crs and scale, to get   
        projection = im.projection().getInfo()['crs']**im.projection().getInfo()['crs']  
        im_reduce = im.reduceRegion(reducer = ee.Reducer.mean(),geometry = ried_225_222,crs=projection,scale = 100, maxPixels= 1e9)    
        Tide up import PAckages   
        Function Section?


## Import Packages
pip install earthengine-python-api??
conda install ipykernel
python -m ipykernel install --user
conda install -c conda-forge geopandas
conda install -c conda-forge ipyleaflet 
conda install -c conda-forge shapely 
conda install -c conda-forge scikit-image

In [1]:
import ee
import ee.mapclient
import datetime
import IPython.display
from IPython.display import Image
import bqplot
import ipywidgets
import pprint
pp = pprint.PrettyPrinter(indent=2) # for printing pretty idk what it is... print with pp.pprint(print stuff)
import pandas as pd
import geopandas as gpd
import matplotlib.dates as mdates
from matplotlib import dates
%matplotlib inline
from shapely.geometry import shape
import skimage
import traitlets
import dateutil.parser
import numpy as np
import ipyleaflet
from ipyleaflet import (LayersControl, basemaps, basemap_to_tiles, LayerGroup, Map, Polygon, GeoJSON)
import json
import os
import requests
# Connect to Earth Engine API
ee.Initialize()

## AOI
[Table of content](#Table-of-content)

In [2]:
# AOI
ried = ee.FeatureCollection('users/tillmueller1990/ried_roi') #Depricated not really Hessisches Ried
ried_225_222 = ee.FeatureCollection('users/tillmueller1990/ried_225_222') # Use this but the boundary isn't that precise at borders, got it from wms layer 
germany = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na','Germany'))

# Create an empty image into which to paint the features, cast to byte.
empty = ee.Image().byte();
# Paint all the polygon edges with the same number and width, display.
germany_outline = empty.paint(germany, 1, 3)

# Datasets 
[Table of content](#Table-of-content)

## Precipitation
[Table of content](#Table-of-content)

In [ ]:
precipitation_gernsheim = ee.FeatureCollection([
            ee.Feature(ee.Geometry.Point(8.489,49.762))
            ])

Map.addLayer(precipitation_gernsheim.draw())
Map.centerObject(precipitation_gernsheim, zoom=12)

## Bare Soil
[Table of content](#Table-of-content)  
https://github.com/brmagnuson/LandFallowingInEarthEngine   


https://github.com/gee-community/gee_tools/blob/master/notebooks/cloud_mask/cloud_masking.ipynb  
https://mygeoblog.com/2019/01/04/harmonizing-sentinel-2-and-landsat-8/   Mergin s2 und landsat

To-Do: Sentinel 2 Cloud mask  
maybe merge all together?  
Make indize bands  
export IC with indize bands as pandas dataframe




In [3]:
from geetools import ui, cloud_mask
from ipygee import *

In [4]:


def get_geometry(ImageCollection):
    geometry = ImageCollection.geometry().getInfo()
    #geometry = [x for x in geometry['coordinates']]
    return geometry

def clip_aoi(ImageCollection):
    im = ImageCollection.clip(ried_225_222)
    return im

def show_tiles(ImageCollection):
    geometry = get_geometry(ImageCollection)
    geometry_list = list(geometry['coordinates'])
    flattened_list = [y for x in geometry_list for y in x] #De flatter list 
    unique_list = []
    unique_list = [x for x in flattened_list if x not in unique_list]
    print("different tiles: ",len(unique_list))
    m = Map(center=(49.6252978589571, 8.34580993652344), zoom=7)
    geo_json = GeoJSON(data=geometry, style = {'color': 'green', 'opacity':1, 'weight':1.9, 'dashArray':'9', 'fillOpacity':0.1})
    m.add_layer(geo_json)
    dc = ipyleaflet.DrawControl()
    m.add_control(dc)
    m.add_control(LayersControl())
    return m
    
def get_properties_l(ImageCollection):
    features = ImageCollection.getInfo()['features']
    dict_list = []
    for f in features:
        prop = f['properties']
        dict_list.append(prop)
    df = pd.DataFrame.from_records(dict_list).drop(['system:footprint'],axis=1)
    #Pandas Series of unique distinc values in df
    unique = df.nunique()
    im_id_list = [item.get('id') for item in ImageCollection.getInfo().get('features')]
    date_list = [datetime.datetime.strptime(x[36:44],'%Y%m%d') for x in im_id_list]
    #property_names = list(df.columns.values) 
    return unique, im_id_list, date_list

def getQABits(image, start, end, mascara):
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start,end+1):
        pattern += 2**i
    # Return a single band image of the extracted QA bits, giving the     band a new name.
    return image.select([0], [mascara]).bitwiseAnd(pattern).rightShift(start)
#A function to mask out cloudy pixels.

def cloudMaskL457(image):
    qa = image.select('pixel_qa');
    #If the cloud bit (5) is set and the cloud confidence (7) is high
    #or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 5).And(qa.bitwiseAnd(1 << 7)).Or(qa.bitwiseAnd(1 << 3))
    #Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min());
    return image.updateMask(cloud.Not()).updateMask(mask2)

def maskL8sr(image):
    #Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5);
    #Get the pixel QA band.
    qa = image.select('pixel_qa')
    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

def maskQuality(image):
    # Select the QA band.
    QA = image.select('pixel_qa')
    # Get the internal_cloud_algorithm_flag bit.
    sombra = getQABits(QA,3,3,'cloud_shadow')
    nubes = getQABits(QA,5,5,'cloud')
    #  var cloud_confidence = getQABits(QA,6,7,  'cloud_confidence')
    cirrus_detected = getQABits(QA,9,9,'cirrus_detected')
    #var cirrus_detected2 = getQABits(QA,8,8,  'cirrus_detected2')
    #Return an image masking out cloudy areas.
    return image.updateMask(sombra.eq(0)).updateMask(nubes.eq(0).updateMask(cirrus_detected.eq(0)))

def maskS2clouds(image):
    qa = image.select('QA60')
    #Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

#Function to mask out every NDVi <0.1 (Water) 
def maskNDVI(image):                   
    ndvi = image.select('NDVI')
    ndvi_mask = ndvi.gte(0.8).And(ndvi.lte(0.1)) 
    return image.updateMask(ndvi_mask)


#https://de.wikipedia.org/wiki/Normalized_Difference_Vegetation_Index
#https://www.linkedin.com/pulse/ndvi-ndbi-ndwi-calculation-using-landsat-7-8-tek-bahadur-kshetri


def NDVI(image):
    ndvi = image.normalizedDifference(['nir','red']) #(first − second) / (first + second)
    return image.addBands(ndvi).rename(image.bandNames().add('NDVI'))

def SAVI(image): #https://de.wikipedia.org/wiki/Soil-Adjusted_Vegetation_Index bodenbereinigter Vegetationsindex (BBVI) 
    #A function to compute Soil Adjusted Vegetation Index."""
    L = 0.5
    savi = ee.Image(0).expression('(1 + L) * float(nir - red)/ (nir + red + L)', {'nir': image.select('nir'), 'red': image.select('red'), 'L': 0.5})
    return image.addBands(savi).rename(image.bandNames().add('SAVI'))

#the normalized difference bare index
def NDBI_swir1(image):
    ndbi = image.normalizedDifference(['swir1', 'nir'])
    return image.addBands(ndbi).rename(image.bandNames().add('NDBI_swir1'))

def NDBI_swir2(image):
    ndbi = image.normalizedDifference(['swir2', 'nir'])
    return image.addBands(ndbi).rename(image.bandNames().add('NDBI_swir2'))

#Built Up Index
def build_up_index(image):
    bu = image.select('NDBI_swir1').subtract(image.select('NDVI'))
    return image.addBands(bu).rename(image.bandNames().add('built_up_index_swir1'))
                                       
                                       
#the normalize difference water index
def ndwi(image):
    return image.normalizedDifference(['B3', 'B8'])


#define thresholds
bareThreshold = -0.32
vegetationThreshold = 0.65
waterThreshold = 0.2

"""
NDVI = -1 to 0 represent Water bodies
NDVI = -0.1 to 0.1 represent Barren rocks, sand, or snow
NDVI = 0.2 to 0.5 represent Shrubs and grasslands or senescing crops
NDVI = 0.6 to 1.0 represent Dense vegetation or tropical rainforest
"""


bandNamesOut_l7 = ['blue','green','red','nir','swir1','temp1','swir2','pixel_qa']
bandNamesOut_l8 = ['ultra blue','blue','green','red','nir','swir1','swir2','temp1','temp2','pixel_qa']
bandNamesOut_s2 = ['Aerosols','blue','green','red','red edge 1','red edge 2','red edge 3','nir','red edge 4','water vapor','cirrus','swir1','swir2','QA60']
bandNamesOut_s2_sr = ['Aerosols','blue','green','red','red edge 1','red edge 2','red edge 3','nir','red edge 4','vater vapor','swir1','swir2','QA60']

bandNamesl7 = ['B1','B2','B3','B4','B5','B6','B7','pixel_qa']
bandNamesl8 = ['B1','B2','B3','B4','B5','B6','B7','B10','B11','pixel_qa']
bandNamesS2 = ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12','QA60']
bandNamesS2_SR  = ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12','QA60']

# Filter the L7 collection to a single month.
l7_sr = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').select(bandNamesl7,bandNamesOut_l7).filterDate(datetime.datetime(2009, 1, 1), datetime.datetime(2019, 6, 1)).filterBounds(ried_225_222).map(clip_aoi).map(cloudMaskL457).map(NDVI).map(SAVI).map(NDBI_swir1).map(NDBI_swir2).map(build_up_index) #https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C01_T1_SR
l8_sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').select(bandNamesl8,bandNamesOut_l8).filterDate(datetime.datetime(2009, 1, 1), datetime.datetime(2019, 6, 1)).filterBounds(ried_225_222).map(clip_aoi).map(maskL8sr).map(NDVI).map(SAVI).map(NDBI_swir1).map(NDBI_swir2).map(build_up_index) #https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR
s2_1c = ee.ImageCollection('COPERNICUS/S2').select(bandNamesS2,bandNamesOut_s2).filterDate(datetime.datetime(2015, 6, 22), datetime.datetime(2017, 3, 27)).filterBounds(ried_225_222).map(clip_aoi).map(maskS2clouds).map(NDVI).map(SAVI).map(NDBI_swir1).map(NDBI_swir2).map(build_up_index) #https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2
s2_sr = ee.ImageCollection('COPERNICUS/S2_SR').select(bandNamesS2_SR,bandNamesOut_s2_sr).filterDate(datetime.datetime(2017, 3, 27), datetime.datetime(2019, 6, 1)).filterBounds(ried_225_222).map(clip_aoi).map(maskS2clouds).map(NDVI).map(SAVI).map(NDBI_swir1).map(NDBI_swir2).map(build_up_index) #https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

pp.pprint(l7_sr.first().date().getInfo())
pp.pprint(l8_sr.first().bandNames().getInfo())
pp.pprint(s2_1c.first().bandNames().getInfo())
pp.pprint(s2_sr.first().bandNames().getInfo())

{'type': 'Date', 'value': 1238407558400}
[ 'ultra blue',
  'blue',
  'green',
  'red',
  'nir',
  'swir1',
  'swir2',
  'temp1',
  'temp2',
  'pixel_qa',
  'NDVI',
  'SAVI',
  'NDBI_swir1',
  'NDBI_swir2',
  'built_up_index_swir1']
[ 'Aerosols',
  'blue',
  'green',
  'red',
  'red edge 1',
  'red edge 2',
  'red edge 3',
  'nir',
  'red edge 4',
  'water vapor',
  'cirrus',
  'swir1',
  'swir2',
  'QA60',
  'NDVI',
  'SAVI',
  'NDBI_swir1',
  'NDBI_swir2',
  'built_up_index_swir1']
[ 'Aerosols',
  'blue',
  'green',
  'red',
  'red edge 1',
  'red edge 2',
  'red edge 3',
  'nir',
  'red edge 4',
  'vater vapor',
  'swir1',
  'swir2',
  'QA60',
  'NDVI',
  'SAVI',
  'NDBI_swir1',
  'NDBI_swir2',
  'built_up_index_swir1']


In [ ]:
visL8 = {'bands':['B5','B6','B4'],'min':0, 'max':5000}
    
visNDVI = {'min': 0, 'max': 1, 'palette': [
        'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163',
        '99B718', '74A901', '66A000', '529400', '3E8601',
        '207401', '056201', '004C00', '023B01', '012E01',
        '011D01', '011301' ]}

visCLC = {'min': 0, 'max': 8, 'palette': [
        'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163',
        '99B718', '74A901', '66A000']}

visS2 = {'min': 0.0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']}



## Corine Land Cover (CLC)

In [3]:
# Load last corine land cover image 
clc = ee.Image.load('COPERNICUS/CORINE/V18_5_1/100m/2012', -1)

# Mask areas where soil moisture measurements valid (farmland cat.:11-16)
clc_mask = clc.gte(11).And(clc.lte(16)) #binary map for updateMask
clc = clc.updateMask(clc_mask) #set mask for not Farmland

# Clip to extend of germany
clc = clc.clip(ried_225_222) #image

# Feature Collection of clc farmland cat.: 11-16 with 350m resolution
clc_vector = clc.reduceToVectors(geometry=ried_225_222, crs=clc.projection(), scale=100, geometryType='polygon', eightConnected = True) #FeatureCollection

#im_reduce = im.reduceRegion(reducer = ee.Reducer.mean(),geometry = ried_225_222,crs=projection,scale = 100, maxPixels= 1e9)    
#Limit to first 5000 Features 
#clc_vector = clc_vector.limit(5000)

#Sample Points Germany 500
random_points = ee.FeatureCollection.randomPoints(clc.geometry(), 5000)

"""
def get_area_from_coordinates(element):
    from shapely.geometry import Polygon
    coords = element
    polygon = Polygon(coords)
    polygon.area
    return 
def get_greatest_feature(FeatureCollection):
    union = FeatureCollection.dissolve()
    geometries = union.geometries()
    for element in geometries:
        print(element)
        
    #geometry = ImageCollection.geometry().getInfo()
    #geometry = [x for x in geometry['coordinates']]
    return geometry

# geometries() --> Returns the list of geometries in a GeometryCollection, or a singleton list of the geometry for single geometries. für ReduceRegions später wichtig
"""

"\ndef get_area_from_coordinates(element):\n    from shapely.geometry import Polygon\n    coords = element\n    polygon = Polygon(coords)\n    polygon.area\n    return \ndef get_greatest_feature(FeatureCollection):\n    union = FeatureCollection.dissolve()\n    geometries = union.geometries()\n    for element in geometries:\n        print(element)\n        \n    #geometry = ImageCollection.geometry().getInfo()\n    #geometry = [x for x in geometry['coordinates']]\n    return geometry\n\n# geometries() --> Returns the list of geometries in a GeometryCollection, or a singleton list of the geometry for single geometries. für ReduceRegions später wichtig\n"

In [ ]:
from geetools import ui, cloud_mask
from ipygee import *
Map = Map()
Map.show()

Map.addLayer(clc_geometry.draw('red'),'clc')




## Precipitaion Data 

In [68]:
#Load Radolan from Assest 
radolan = ee.ImageCollection('users/tillmueller1990/radolan/radolan_dwd_germany')

#Load GSMAP Hourly Precipitation
gsmap = ee.ImageCollection('JAXA/GPM_L3/GSMaP/v6/operational')

#CHIRPS 5km daily precipitation 
chirps  = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')

## Soil Moisture

Products provided by NASA, ESA, GEE ....

In [14]:
#NASA-USDA Global Soil Moisture Data / 3 days / 2010- 2019 / 0.25 arc degrees / SMOS lvl2 integrated into two-layer palmer model
SMOS = ee.ImageCollection('NASA_USDA/HSL/soil_moisture')
surface_moisture = SMOS.select('ssm') #units mm, min=0 max=25
subsurface_moisture = SMOS.select('susm') #units: mm min=0, max=275
moisture_profile = SMOS.select('smp') #units: fraction, min0, max 1

#NASA-USDA SMAP Global Soil Moisture Data / 3 days / 2015 - 2019 / 0.25 arc degrees / SMAP level 3 + two-layer Palmer
SMAP = ee.ImageCollection('NASA_USDA/HSL/SMAP_soil_moisture')

#GLDAS-2.1: Global Land Data Assimilation System / 3 hours / 0.25 arc degrees / 2000 - 2019 / 

## GLDAS-2.1: Global Land Data Assimilation System 
Global Land Data Assimilation System (GLDAS) ingests satellite and ground-based observational data products. Using advanced land surface modeling and data assimilation techniques, it generates optimal fields of land surface states and fluxes.

In [75]:
GLDAS = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H')


# Sentinel1 derives soil-moisture
[Table of content](#Table-of-content)

In [60]:
#Sentinel 1 backscatter raw data convertet to reflectivity range between 0 and 1 indicator for high reflectivity = high water volume content and vis verse 
#Load Sentinel 1 and filter data
def load_dataset(ImageCollection_ID,begin,end,aoi):
    ic = ee.ImageCollection(ImageCollection_ID).filterDate(begin,end).filterBounds(aoi)
    return ic

def filter_sentinel1(ImageCollection,polarisation,instrumentMode,resolution):
    ic = ImageCollection.filter(ee.Filter.listContains('transmitterReceiverPolarisation',polarisation)).filter(ee.Filter.eq('instrumentMode',instrumentMode)).filterMetadata('resolution_meters','equals', resolution)
    return ic

def seperate_look_angels(ImageCollection):
    Ascending = ImageCollection.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    Descending = ImageCollection.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
    return Ascending,Descending

def seperate_tiles(ImageCollection,tiles):
    tile_list = [x]

def get_geometry(ImageCollection):
    geometry = ImageCollection.geometry().getInfo()
    #geometry = [x for x in geometry['coordinates']]
    return geometry

def show_tiles(ImageCollection):
    geometry = get_geometry(ImageCollection)
    geometry_list = list(geometry['coordinates'])
    flattened_list = [y for x in geometry_list for y in x] #De flatter list 
    unique_list = []
    unique_list = [x for x in flattened_list if x not in unique_list]
    print("different tiles: ",len(unique_list))
    m = Map(center=(49.6252978589571, 8.34580993652344), zoom=7)
    geo_json = GeoJSON(data=geometry, style = {'color': 'green', 'opacity':1, 'weight':1.9, 'dashArray':'9', 'fillOpacity':0.1})
    m.add_layer(geo_json)
    dc = ipyleaflet.DrawControl()
    m.add_control(dc)
    m.add_control(LayersControl())
    return m
    
def get_properties(ImageCollection):
    features = ImageCollection.getInfo()['features']
    dict_list = []
    for f in features:
        prop = f['properties']
        dict_list.append(prop)
    df = pd.DataFrame.from_records(dict_list).drop(['system:footprint','transmitterReceiverPolarisation'],axis=1)
    #Pandas Series of unique distinc values in df
    unique = df.nunique()
    im_id_list = [item.get('id') for item in ImageCollection.getInfo().get('features')]
    date_list = [datetime.datetime.strptime(x[35:43],'%Y%m%d') for x in im_id_list]
    #property_names = list(df.columns.values) 
    return unique, im_id_list, date_list

def make_mosaic(date,ImageCollection):
    date = ee.Date(date['value'])
    filterCollection = ImageCollection.filterDate(date, date.advance(1,'day'))
    #Make the mosaic
    image = ee.Image(filterCollection.mosaic()).copyProperties(filterCollection.first(),["system:time_start"])
    #Add the mosaic to a list only if the collection has images
    #return ee.List(ee.Algorithms.If(filterCollection.size(), newList.add(image), newList))
    return image

def clip_aoi(ImageCollection):
    im = ImageCollection.clip(ried_225_222)
    return im

def add_area(image):
    area = image.multiply(ee.Image.pixelArea()).divide(-(1000*1000))
    stat = area.reduceRegion(ee.Reducer.sum(),ried_225_222,10) 
    im = image.set('area',stat.get('VV'))
    return im

def reproject(image):
    VV = image.select('VV')
    return image.reporject({crs: VV.projection().crs(), scale : 100})

def toNatural(image):
    return ee.Image(10.0).pow(image.select(0).divide(10.0)).copyProperties(image)

def toDB(image):
    return ee.Image(image).log10().multiply(10.0).copyProperties(image)

#Sigma Lee speckle filteringThe RL speckle filter from https://code.earthengine.google.com/2ef38463ebaf5ae133a478f173fd0ab5 by Guido Lemoine
def RefinedLee(img):
    #img must be in natural units, i.e. not in dB!
    #Set up 3x3 kernels
    weights3 = ee.List.repeat(ee.List.repeat(1,3),3)
    kernel3 = ee.Kernel.fixed(3,3, weights3, 1, 1, False)
    mean3 = img.reduceNeighborhood(ee.Reducer.mean(), kernel3)
    variance3 = img.reduceNeighborhood(ee.Reducer.variance(), kernel3)
    #Use a sample of the 3x3 windows inside a 7x7 windows to determine gradients and directions
    sample_weights = ee.List([[0,0,0,0,0,0,0], [0,1,0,1,0,1,0],[0,0,0,0,0,0,0], [0,1,0,1,0,1,0], [0,0,0,0,0,0,0], [0,1,0,1,0,1,0],[0,0,0,0,0,0,0]])
    sample_kernel = ee.Kernel.fixed(7,7, sample_weights, 3,3, False)
    #Calculate mean and variance for the sampled windows and store as 9 bands
    sample_mean = mean3.neighborhoodToBands(sample_kernel)
    sample_var = variance3.neighborhoodToBands(sample_kernel)
    #Determine the 4 gradients for the sampled windows
    gradients = sample_mean.select(1).subtract(sample_mean.select(7)).abs()
    gradients = gradients.addBands(sample_mean.select(6).subtract(sample_mean.select(2)).abs())
    gradients = gradients.addBands(sample_mean.select(3).subtract(sample_mean.select(5)).abs())
    gradients = gradients.addBands(sample_mean.select(0).subtract(sample_mean.select(8)).abs())
    #And find the maximum gradient amongst gradient bands
    max_gradient = gradients.reduce(ee.Reducer.max())
    #Create a mask for band pixels that are the maximum gradient
    gradmask = gradients.eq(max_gradient)
    #duplicate gradmask bands: each gradient represents 2 directions
    gradmask = gradmask.addBands(gradmask)
    #Determine the 8 directions
    directions = sample_mean.select(1).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(7))).multiply(1)
    directions = directions.addBands(sample_mean.select(6).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(2))).multiply(2))
    directions = directions.addBands(sample_mean.select(3).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(5))).multiply(3))
    directions = directions.addBands(sample_mean.select(0).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(8))).multiply(4))
    #The next 4 are the not() of the previous 4
    directions = directions.addBands(directions.select(0).Not().multiply(5))
    directions = directions.addBands(directions.select(1).Not().multiply(6))
    directions = directions.addBands(directions.select(2).Not().multiply(7))
    directions = directions.addBands(directions.select(3).Not().multiply(8))
    #Mask all values that are not 1-8
    directions = directions.updateMask(gradmask)
    #"collapse" the stack into a singe band image (due to masking, each pixel has just one value (1-8) in it's directional band, and is otherwise masked)
    directions = directions.reduce(ee.Reducer.sum())
    #Generate stats
    sample_stats = sample_var.divide(sample_mean.multiply(sample_mean))
    #Calculate localNoiseVariance
    sigmaV = sample_stats.toArray().arraySort().arraySlice(0,0,5).arrayReduce(ee.Reducer.mean(), [0])
    #Set up the 7*7 kernels for directional statistics
    rect_weights = ee.List.repeat(ee.List.repeat(0,7),3).cat(ee.List.repeat(ee.List.repeat(1,7),4))
    #Set weights
    diag_weights = ee.List([[1,0,0,0,0,0,0], [1,1,0,0,0,0,0], [1,1,1,0,0,0,0],[1,1,1,1,0,0,0], [1,1,1,1,1,0,0], [1,1,1,1,1,1,0], [1,1,1,1,1,1,1]])
    rect_kernel = ee.Kernel.fixed(7,7, rect_weights, 3, 3, False)
    diag_kernel = ee.Kernel.fixed(7,7, diag_weights, 3, 3, False)
    #Create stacks for mean and variance using the original kernels Mask with relevant direction.
    dir_mean = img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel).updateMask(directions.eq(1))
    dir_var = img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel).updateMask(directions.eq(1))
    dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(),diag_kernel).updateMask(directions.eq(2)))
    dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(),diag_kernel).updateMask(directions.eq(2)))
    #and add the bands for rotated kernels
    for i in range(4):
        dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(),rect_kernel.rotate(i)).updateMask(directions.eq(2*i+1)))
        dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(),rect_kernel.rotate(i)).updateMask(directions.eq(2*i+1)))
        dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(),diag_kernel.rotate(i)).updateMask(directions.eq(2*i+2)))
        dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(),diag_kernel.rotate(i)).updateMask(directions.eq(2*i+2)))
    #"collapse" the stack into a single band image (due to masking, each pixel has just one value in it's directional band, and is otherwise masked)
    dir_mean = dir_mean.reduce(ee.Reducer.sum())
    dir_var = dir_var.reduce(ee.Reducer.sum())
    #A finally generate the filtered value
    varX = dir_var.subtract(dir_mean.multiply(dir_mean).multiply(sigmaV)).divide(sigmaV.add(1.0))
    b = varX.divide(dir_var)
    result = dir_mean.add(b.multiply(img.subtract(dir_mean)))
    return result.arrayFlatten([['VV']]).copyProperties(img)

def calc_asc_soilMoisture(image):
    im = image.expression('(omega - omegaD) / (omegaW - omegaD)', {'omega' : image.select('VV'), 'omegaD' : minMax_asc.select('VV_max'), 'omegaW' : minMax_asc.select('VV_min')})
    return im.addBands(im).select(['VV','VV_1'],['Backscatter_Coefficient','soil_moisture_content'])

def calc_des_soilMoisture(image):
    im = image.expression('(omega - omegaD) / (omegaW - omegaD)', {'omega' : image.select('VV'), 'omegaD' : minMax_des.select('VV_max'), 'omegaW' : minMax_des.select('VV_min')})
    return im.addBands(im).select(['VV','VV_1'],['Backscatter_Coefficient','soil_moisture_content'])

def filter_IC(ImageCollection,filter):
    old_size = ImageCollection.size().getInfo()
    new_coll = ImageCollection.filter(filter)
    new_size = new_coll.size().getInfo()
    return new_coll

def reducer(ImageCollection,reducer):
    im = ImageCollection.reduce(reducer)
    return im

def plot_image(ImageCollection):
    m = Map(center=(49.6252978589571, 8.34580993652344), zoom=7)
    ic = GetTileLayerUrl(ImageCollection.first().visualize())
    m.add_layer(ic)
    dc = ipyleaflet.DrawControl()
    m.add_control(dc)
    m.add_control(LayersControl())
    return m

def windy_days_filter(image):
    d = image.date().format('Y-M-d')
    wx = ee.ImageCollection('NOAA/CFSV2/FOR6H')
    vWind = wx.select(['v-component_of_wind_height_above_ground'])
    a = vWind.max()
    uWind = wx.select(['u-component_of_wind_height_above_ground'])
    b = uWind.max()
    a = a.pow(2)
    b = b.pow(2)
    ab = a.add(b)
    ws = ab.sqrt()
    ws = ws.multiply(3.6)
    return image.updateMask(ws.lt(12))

#Time of interest
begin = ee.Date.fromYMD(2013,1,1)
end = ee.Date.fromYMD(2019,6,1)
date_range = end.difference(begin, 'day')

#Source dataset
sentinel1 = load_dataset('COPERNICUS/S1_GRD',begin,end,ried_225_222)
print("sentinel1",type(sentinel1),"Collection Size: ", sentinel1.size().getInfo())

#Filter dataset for High resolution and Vertical transmitt vertical receive
sentinel1_VV = filter_sentinel1(sentinel1,'VV','IW',10)
print("sentinel1_VV",type(sentinel1_VV),"Collection Size: ", sentinel1_VV.size().getInfo())

print(sentinel1_VV.first().propertyNames().getInfo())
#The RL speckle filter from https://code.earthengine.google.com/2ef38463ebaf5ae133a478f173fd0ab5 by Guido Lemoine
sentinel1_VV_natural = sentinel1_VV.map(toNatural)
#print(sentinel1_VV_natural.first().propertyNames().getInfo())
sentinel1_VV_slFilter = sentinel1_VV_natural.map(RefinedLee)
#print(sentinel1_VV_slFilter.first().propertyNames().getInfo())                      
sentinel1_VV = sentinel1_VV_slFilter.map(toDB)
#print(sentinel1_VV.first().bandNames().getInfo())
#print(sentinel1_VV.first().propertyNames().getInfo())

#Filter for different look angles
VV_Ascending,VV_Descending = seperate_look_angels(sentinel1_VV)
print("VV_Ascending",type(VV_Ascending),"VV_Descending",type(VV_Descending),"Collection Size: ", VV_Ascending.size().getInfo(), VV_Descending.size().getInfo())

#Clip images to AOI and calculate area property
VV_aoi_asc = VV_Ascending.map(clip_aoi).map(add_area)
print("VV_aoi_asc",type(VV_aoi_asc),"Collection Size: ", VV_aoi_asc.size().getInfo())
VV_aoi_des = VV_Descending.map(clip_aoi).map(add_area)
print("VV_aoi_des",type(VV_aoi_des),"Collection Size: ", VV_aoi_des.size().getInfo())

#Create Min and Max bands for change detection method
minMax_asc = reducer(VV_aoi_asc,ee.Reducer.minMax())
print("minMax_asc",type(minMax_asc),minMax_asc.getInfo())
minMax_des = reducer(VV_aoi_des,ee.Reducer.minMax())
print("minMax_des",type(minMax_des),minMax_des.getInfo())

#Compute soil moisture with simple change detection Methode
VV_asc_sm = VV_aoi_asc.map(calc_asc_soilMoisture)
print("VV_asc_sm",type(VV_asc_sm),"Collection Size: ", VV_asc_sm.size().getInfo())
VV_des_sm = VV_aoi_des.map(calc_des_soilMoisture)
print("VV_des_sm",type(VV_des_sm),"Collection Size: ", VV_des_sm.size().getInfo())



"""
#Get list of ids,dates and unique count of prop
unique, im_id_list, date_list = get_properties(VV_Ascending)
date_list = ee.List([ee.Date(x) for x in date_list])
#pp.pprint(unique)

#Improve dataset validity 
VV_aoi_filtered = filter_IC(VV_aoi_area,ee.Filter.gte('area',250))
print("VV_aoi_filtered",type(VV_aoi_filtered),"Collection Size: ", VV_aoi_filtered.size().getInfo())

pp.pprint(VV_des_sm.getInfo().get('features')[1])
"""




sentinel1 <class 'ee.imagecollection.ImageCollection'> Collection Size:  1160
sentinel1_VV <class 'ee.imagecollection.ImageCollection'> Collection Size:  1128
['GRD_Post_Processing_start', 'sliceNumber', 'system:id', 'GRD_Post_Processing_facility_name', 'resolution', 'SLC_Processing_facility_name', 'system:footprint', 'familyName', 'system:version', 'segmentStartTime', 'missionDataTakeID', 'GRD_Post_Processing_facility_country', 'nssdcIdentifier', 'productClass', 'phaseIdentifier', 'orbitProperties_pass', 'relativeOrbitNumber_stop', 'system:time_end', 'SLC_Processing_facility_site', 'GRD_Post_Processing_stop', 'system:time_start', 'instrumentMode', 'totalSlices', 'SLC_Processing_stop', 'startTimeANX', 'SLC_Processing_start', 'resolution_meters', 'instrumentSwath', 'relativeOrbitNumber_start', 'productTimelinessCategory', 'SLC_Processing_software_name', 'sliceProductFlag', 'S1TBX_Calibration_vers', 'orbitNumber_start', 'GRD_Post_Processing_facility_site', 'instrument', 'GRD_Post_Process

'\n#Get list of ids,dates and unique count of prop\nunique, im_id_list, date_list = get_properties(VV_Ascending)\ndate_list = ee.List([ee.Date(x) for x in date_list])\n#pp.pprint(unique)\n\n#Improve dataset validity \nVV_aoi_filtered = filter_IC(VV_aoi_area,ee.Filter.gte(\'area\',250))\nprint("VV_aoi_filtered",type(VV_aoi_filtered),"Collection Size: ", VV_aoi_filtered.size().getInfo())\n\npp.pprint(VV_des_sm.getInfo().get(\'features\')[1])\n'

## Get CSV for Datasets
[Table of content](#Table-of-content)  
Only year by year otherwise memory exceeded

In [5]:
aoi = ee.Geometry.Polygon([[[8.492535113992176,49.806727648654025],[8.474855734199537,49.8022389698566],[8.529615838732752,49.758455050463446],[8.562403236731484,49.777747019690885],[8.518286161973151,49.805895076244035],[8.492535113992176,49.806727648654025]]])

### Get CSV from SMOS soil-moisture for Ried (1 Polygon)

In [67]:
#Geometrys, Regions, Source Dataset, TOI, 
# Function to iterate over image collection, returning a pandas dataframe
start,end = datetime.datetime(2014,1,1),datetime.datetime(2020,1,1)
sm = SMOS.filterDate(start,end)

def extract_point_values(img_id):
    IC = sm.filter(ee.Filter.eq('system:index',img_id))
    image = IC.reduce(ee.Reducer.first())
    #Ad reducer output to the Features in the collection.
    #projection = im.projection().getInfo()['crs']
    im_reduce = image.reduceRegion(ee.Reducer.mean(),aoi, scale = 100) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = im_reduce.getInfo()
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item['properties']['system:index'] for item in sm.getInfo().get('features')]
im_date = [datetime.datetime.strptime(x[12:20],'%Y%m%d') for x in im_id] #http://joda-time.sourceforge.net/apidocs/org/joda/time/format/DateTimeFormat.html

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0])
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("SMOS_testGeometry.csv",index=False)

### Get CSV for S1 derives Soil Moisture and Backscatter Signal for Ried (1 Polygon)

To-D:  
Backscatter Coefficient ist gleich dem content der Bodenfeuchte?  

In [61]:
#Geometrys, Regions, Source Dataset, TOI, 
# Function to iterate over image collection, returning a pandas dataframe
sm = VV_asc_sm
def extract_point_values(img_id):
    IC = sm.filter(ee.Filter.eq('system:index',img_id))
    image = IC.reduce(ee.Reducer.first())
    #Ad reducer output to the Features in the collection.
    #projection = im.projection().getInfo()['crs']
    im_reduce = image.reduceRegion(ee.Reducer.mean(),aoi, scale = 100) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = im_reduce.getInfo()
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item['properties']['system:index'] for item in sm.getInfo().get('features')]
im_date = [datetime.datetime.strptime(x[33:48],'%Y%m%dT%H%M%S') for x in im_id] #http://joda-time.sourceforge.net/apidocs/org/joda/time/format/DateTimeFormat.html

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0])
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("S1_asc_100_testGeometry_LF.csv",index=False)

### Get CSV for Chrisp derived precipitation for Ried (1 Polygon)

In [69]:
#Geometrys, Regions, Source Dataset, TOI, 
start,end = datetime.datetime(2014,1,1),datetime.datetime(2020,1,1)
#aoi = ried
sm = chirps.filterDate(start,end)

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, aoi):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegion(ee.Reducer.mean(), aoi) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = fc_image_red.getInfo()
    data_aoi['geometry'] = aoi.getInfo()['features']
    data_aoi['date'] = datetime.datetime.strptime(image.getInfo()['id'][22:31],'%Y%m%d')
    df = pd.DataFrame(data_aoi)
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item.get('id') for item in sm.getInfo().get('features')]

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0], aoi)
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i, aoi))
df_all.to_csv("chirps_testGeometry.csv",index=False)


KeyError: 'features'

In [73]:
#Geometrys, Regions, Source Dataset, TOI, 
# Function to iterate over image collection, returning a pandas dataframe
start,end = datetime.datetime(2014,1,1),datetime.datetime(2020,1,1)
sm = chirps.filterDate(start,end)

def extract_point_values(img_id):
    IC = sm.filter(ee.Filter.eq('system:index',img_id))
    image = IC.reduce(ee.Reducer.first())
    #Ad reducer output to the Features in the collection.
    #projection = im.projection().getInfo()['crs']
    im_reduce = image.reduceRegion(ee.Reducer.mean(),aoi, scale = 100) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = im_reduce.getInfo()
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item['properties']['system:index'] for item in sm.getInfo().get('features')]
im_date = [datetime.datetime.strptime(x[0:8],'%Y%m%d') for x in im_id] #http://joda-time.sourceforge.net/apidocs/org/joda/time/format/DateTimeFormat.html

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0])
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("SMOS_testGeometry.csv",index=False)

['20140101', '20140102', '20140103', '20140104', '20140105', '20140106', '20140107', '20140108', '20140109', '20140110', '20140111', '20140112', '20140113', '20140114', '20140115', '20140116', '20140117', '20140118', '20140119', '20140120', '20140121', '20140122', '20140123', '20140124', '20140125', '20140126', '20140127', '20140128', '20140129', '20140130', '20140131', '20140201', '20140202', '20140203', '20140204', '20140205', '20140206', '20140207', '20140208', '20140209', '20140210', '20140211', '20140212', '20140213', '20140214', '20140215', '20140216', '20140217', '20140218', '20140219', '20140220', '20140221', '20140222', '20140223', '20140224', '20140225', '20140226', '20140227', '20140228', '20140301', '20140302', '20140303', '20140304', '20140305', '20140306', '20140307', '20140308', '20140309', '20140310', '20140311', '20140312', '20140313', '20140314', '20140315', '20140316', '20140317', '20140318', '20140319', '20140320', '20140321', '20140322', '20140323', '20140324', '20

### Get CSV for GLDAS climate values for Ried (1 Polygon)

In [ ]:
#Geometrys, Regions, Source Dataset, TOI, 
start,end = datetime.datetime(2014,1,1),datetime.datetime(2020,1,1)
#aoi = ried
sm = GLDAS.filterDate(start,end)

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, aoi):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegion(ee.Reducer.mean(), aoi) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = fc_image_red.getInfo()
    #data_aoi['date'] = datetime.datetime.strptime(image.getInfo()['id'][31:39],'%Y%m%d')
    #df = pd.DataFrame(data_aoi,columns=['Albedo_inst', 'AvgSurfT_inst', 'CanopInt_inst', 'ECanop_tavg', 'ESoil_tavg', 'Evap_tavg', 'LWdown_f_tavg', 'Lwnet_tavg', 'PotEvap_tavg', 'Psurf_f_inst', 'Qair_f_inst', 'Qg_tavg', 'Qh_tavg', 'Qle_tavg', 'Qs_acc', 'Qsb_acc', 'Qsm_acc', 'Rainf_f_tavg', 'Rainf_tavg', 'RootMoist_inst', 'SWE_inst', 'SWdown_f_tavg', 'SnowDepth_inst', 'Snowf_tavg', 'SoilMoi0_10cm_inst', 'SoilMoi100_200cm_inst', 'SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilTMP0_10cm_inst', 'SoilTMP100_200cm_inst', 'SoilTMP10_40cm_inst', 'SoilTMP40_100cm_inst', 'Swnet_tavg', 'Tair_f_inst', 'Tveg_tavg', 'Wind_f_inst'])
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item.get('id') for item in sm.getInfo().get('features')]
im_date = [datetime.datetime.strptime(x[31:44],'%Y%m%d_%H%M') for x in im_id]

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0], aoi)
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i, aoi))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("GLDAS_testGeometry.csv",index=False)

In [80]:
#Geometrys, Regions, Source Dataset, TOI, 
# Function to iterate over image collection, returning a pandas dataframe
start,end = datetime.datetime(2014,1,1),datetime.datetime(2020,1,1)
sm = GLDAS.filterDate(start,end).filterMetadata('start_hour', 'equals', 18) #Daten für 18 Uhr if not than you have more than 5000 Elements

def extract_point_values(img_id):
    IC = sm.filter(ee.Filter.eq('system:index',img_id))
    image = IC.reduce(ee.Reducer.first())
    #Ad reducer output to the Features in the collection.
    #projection = im.projection().getInfo()['crs']
    im_reduce = image.reduceRegion(ee.Reducer.mean(),aoi, scale = 100) #,scale=1000
    # Convert to Pandas Dataframe
    data_aoi = im_reduce.getInfo()
    df = pd.DataFrame.from_records([data_aoi])
    return df

#List of image propertys 
im_prop = sm.first().propertyNames().getInfo()
#Get propertie from every image to a List
im_id = [item['properties']['system:index'] for item in sm.getInfo().get('features')]
#print(im_id)
im_date = [datetime.datetime.strptime(x[1:12],'%Y%m%d_%H') for x in im_id] #http://joda-time.sourceforge.net/apidocs/org/joda/time/format/DateTimeFormat.html

#### Create Initial Pandas Dataframe
df_all = extract_point_values(im_id[0])
df_all = df_all.drop([0])

#### Iterate over all impages
for i in im_id:
    df_all = df_all.append(extract_point_values(i))

#Set date and save a lot of .getInfo() calls
df_all['date'] = im_date

df_all.to_csv("GLDAS_testGeometry.csv",index=False)

# Data Visualisation  
[Table of content](#Table-of-content)  

In [ ]:
#Get TileLayerurl from ee to plot on ipyleaflet
def GetTileLayerUrl(ee_image_object):
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return tile_url_template.format(**map_id)

In [ ]:
#Visualization Parameters
dark_matter_layer = basemap_to_tiles(basemaps.CartoDB.DarkMatter)
germany_viz = GetTileLayerUrl(germany_outline.visualize())
#radolan_viz = GetTileLayerUrl(radolan.visualize(min=0,max=40,bands=['b1']))
clc_viz = GetTileLayerUrl(clc.visualize(min=0,max=7,palette=['FFE6FF','FFFFA8','FFFF00','E6E600','E68000','F2A64D','E6A600']))
#clc_vector_viz = GetTileLayerUrl(clc_vector.draw(color='red').visualize())
#surface_moisture_viz = GetTileLayerUrl(surface_moisture.visualize(min=0,max=28,palette=['0300ff', '418504', 'efff07', 'efff07', 'ff0303']))
#random_points_viz = GetTileLayerUrl(random_points.draw(color='blue').visualize())
#sentinel1_viz = GetTileLayerUrl(newcol.first().visualize())

# Create layer group
layer_group = LayerGroup(layers=(ipyleaflet.TileLayer(url=clc_viz,name='Corine Land Cover 2012'),
                                 ipyleaflet.TileLayer(url=germany_viz, name='german boundary'),
                                 #ipyleaflet.TileLayer(url=clc_vector_viz, name='clc_vector')
                                 #ipyleaflet.TileLayer(url=random_points_viz, name='rand point')
                                 #ipyleaflet.TileLayer(url=surface_moisture_viz, name='GSMD surface moisture')
                                ))
#Map options
center,zoom = (49.6252978589571, 8.34580993652344),11

#Interactive Visualizations
map1 = ipyleaflet.Map(layer= dark_matter_layer, center=center, zoom=zoom, layout={'height' : '600px'})
dc = ipyleaflet.DrawControl()
map1.add_control(dc)
map1.add_layer(layer_group)
map1.add_control(LayersControl())
map1


In [ ]:
#Get Coordinates from Drawn Geometry
dc.last_draw

# Collection of commands   
[Table of content](#table-of-content)

In [ ]:
#Command Collection
.size().getInfo() # Get collection size
.bandNames().getInfo() # Get List with alls Bands from ee.Image not ImageCollection
.geometry().bounds().getInfo() # get Geometry of a Feature // Get bounding box of this geometry
.limit(5000) # Limit to the first 5000 Elements/Features
.normalizedDifference(bandNames) #(first − second) / (first + second) Indice
image.addBands(ndvi).rename(image.bandNames().add('NDVI')) #add band to the image and rename it correctly

#Vectorizing
var vectors = image.reduceToVectors({
  geometry: FeatureCollection,
  crs: image.projection(),
  scale: 1000,
  geometryType: 'polygon',
  eightConnected: false,
  labelProperty: 'zone',
  reducer: ee.Reducer.mean()
});

#Masking
.clip(feature)
var image = ee.Image
var mask = image.gte(2).And(lt(5))
var maskedImage = image.updateMask(mask)

#Visualizations
thumbnail_url = image.getThumbUrl({
    'bands' : '',
    'min' : ,
    'max' : ,
    'region' : .geometry().bounds().getInfo() #must be a geojson 
})
IPython.display.HTML('Thumnail URL: <a href={0}>{0}</a>'.format(thumbnail_url)) #create url to view
IPython.display.Image(url=thumbnail_url) # view direct in notebook

#Interactive Visualizations
import ipyleaflet
map1 = ipyleaflet.Map(zoom=3, layout={'height' : '400px'})
dc = ipyleaflet.DrawControl()
map1.add_control(dc)
map1
dc.last_draw # gives information about the last drawn polygon (coordinates etc.)

#Function to create a tile layer urlfrom an gee image object
def GetTileLayerUrl(ee_image_object):
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return tile_url_template.format(**map_id)

#style the image
tile_url = GetTileLayerUrl(image.visualize(min=0, max=3000, gamma=1.5, bands=['','','']))
map1.add_layer(ipyleaflet.TileLayer(url=tile_url))
#or create layer groups 

# Collection of Notebook snippets 
[Table of content](#Table-of-content)  
for getting pandas DataFrame of AOI Time Series Satellite Data from GEE

In [ ]:
#Geometrys, Regions
points = ee.FeatureCollection([
            ee.Feature(ee.Geometry.Point(8.234870250943459,49.80831604635797)),
            ee.Feature(ee.Geometry.Point(8.241221721890724,49.6585087644599)),
            ])
point = ee.Feature(ee.Geometry.Point(8.234870250943459,49.80831604635797))

#Time
start,end = datetime.datetime(2015,1,1),datetime.datetime(2016,1,1)
#Source Dataset
chirps = chirps_precipitation.filterDate(start,end)
# Function Convert a FeatureCollection into a pandas DataFrame; Features is a list of dict with the output
def fc2df(fc):
    # Convert a FeatureCollection into a pandas DataFrame
    # Features is a list of dict with the output
    features = fc.getInfo()['features']

    dictarr = []
    for f in features:
        # Store all attributes in a dict
        attr = f['properties']
        # and treat geometry separately
        attr['geometry'] = f['geometry']  # GeoJSON Feature!
        #attr['geometrytype'] = f['geometry']['type']
        dictarr.append(attr)

    df = gpd.GeoDataFrame(dictarr)
    # Convert GeoJSON features to shape
    #df['geometry'] = map(lambda x: shape(x), df.geometry)    
    return df

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, pts):
    image = ee.Image(img_id)
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegions(collection=pts, reducer=ee.Reducer.mean(), scale=1000)
    # Convert to Pandas Dataframe
    df_image_red = fc2df(fc_image_red)
    # Add Date as Variable
    df_image_red['date'] = datetime.datetime.strptime(image.getInfo()['id'][22:31],'%Y%m%d')
    print(df_image_red)
    return df_image_red

#List of image propertys 
chirps_prop = surface_moisture.first().propertyNames().getInfo()
print('Check 1')
#Get propertie from every image to a List
chirps_id = [] #empty list
chirps_id = [item.get('id') for item in chirps.getInfo().get('features')]
#### Create Initial Pandas Dataframe
df_all = extract_point_values(chirps_id[0], points)
df_all = df_all.drop([0,1])
print("check 3")
#### Iterate over all impages
c=0
for i in chirps_id:
    c = c+ 1
    print("c",c)
    df_all = df_all.append(extract_point_values(i, points))
print("check 4")
#### Display Results
pp.pprint(df_all)
rad_point1 = df_all.loc[0]
rad_point2 = df_all.loc[1]


# https://gis.stackexchange.com/questions/313186/extracting-pixel-time-series-from-google-earth-engine

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import ee
from ipygee import *


def extract_time_series(start,end,coll,ried): #,sf

    
    # Obtain image collection for all images within query dates
    coll = coll.filterDate(start,end)

    # Get list of images which correspond with the above
    images = [item.get('id') for item in coll.getInfo().get('features')]
    store = []
    date_store = []
    print(images)
    # Loop over all images and extract pixel value
    for image in images:
        
        im = ee.Image(image)
        #projection = im.projection().getInfo()['crs']
        # Obtain date from timestamp in metadata
        date = dt.fromtimestamp(im.get("system:time_start").getInfo() / 1000.)
        date_store.append(np.datetime64(date))

        # Extract pixel value
        data = im.reduceRegion(ee.Reducer.mean(),ried, 1000) #,1, crs=projection).get(band_name) 
        store.append(data.getInfo())
        print(store)
    # Scale the returned data based on scale factor
    #store = [x * sf if isinstance(x, int) else np.nan for x in store]
    
    # Convert output into pandas data frame
    df = pd.DataFrame(index=date_store, data=store, columns=['precipitation'])
    df['store']
    return df


band_name = 'b1'
coll = chirps_precipitation
start,end = datetime.datetime(2015,1,1),datetime.datetime(2015,1,5)
# Set up point geometry
points = ee.FeatureCollection([
        ee.Feature(ee.Geometry.Point(8.234870250943459,49.80831604635797)),
        ee.Feature(ee.Geometry.Point(8.241221721890724,49.6585087644599)),
        ])
point = ee.Feature(ee.Geometry.Point(8.234870250943459,49.80831604635797))


rad_data = extract_time_series(start,end,coll,ried)
print(rad_data)